In [ ]:
!pip3 install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.1/241.1 MB 4.3 MB/s eta 0:00:0000:0100:01
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.7/979.7 kB 15.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 10.8 MB/s eta 0:00:0000:0100:01
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached libclang-14.0.6-py2.py3-none-macosx_10_9_x86_64.whl (13.2 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:0000:0100:01
  Using cached termcolor-2.0.1-py3-none-any.whl (5.4 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached keras-2.

In [ ]:
!pip install icrawler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.1 MB/s eta 0:00:0000:0100:01


In [3]:
from icrawler.builtin import BingImageCrawler

# 猫の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "cat"})
crawler.crawl(keyword="猫", max_num=100)

from icrawler.builtin import BingImageCrawler

# 犬の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "dog"})
crawler.crawl(keyword="犬", max_num=100)
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile
# IOError: image file is truncated (0 bytes not processed)回避のため
ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                # FLIP_LEFT_RIGHT　は 左右反転
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index)


X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./dog_cat.npy", xy)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np
from tensorflow.keras import optimizers

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64
print(X_train.shape)

def load_data():
    X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)
    # 入力データの各画素値を0-1の範囲で正規化(学習コストを下げるため)
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    # to_categorical()にてラベルをone hot vector化
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test

def train(X, y, X_test, y_test):
    model = Sequential()

    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = optimizers.RMSprop(lr=0.00005, decay=1e-6)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)
    # HDF5ファイルにKerasのモデルを保存
    model.save('./cnn.h5')

    return model

def main():
    # データの読み込み
    X_train, y_train, X_test, y_test = load_data()

    # モデルの学習
    model = train(X_train, y_train, X_test, y_test)

main()
print("we finished training classification model")

2022-10-08 12:59:15,415 - INFO - icrawler.crawler - start crawling...
2022-10-08 12:59:15,420 - INFO - icrawler.crawler - starting 1 feeder threads...
2022-10-08 12:59:15,425 - INFO - feeder - thread feeder-001 exit
2022-10-08 12:59:15,430 - INFO - icrawler.crawler - starting 1 parser threads...
2022-10-08 12:59:15,439 - INFO - icrawler.crawler - starting 1 downloader threads...
2022-10-08 12:59:16,035 - INFO - parser - parsing result page https://www.bing.com/images/async?q=猫&first=0
2022-10-08 12:59:16,081 - INFO - downloader - skip downloading file 000001.jpg
2022-10-08 12:59:16,083 - INFO - downloader - skip downloading file 000002.jpg
2022-10-08 12:59:16,083 - INFO - downloader - skip downloading file 000003.jpg
2022-10-08 12:59:16,085 - INFO - downloader - skip downloading file 000004.jpg
2022-10-08 12:59:16,086 - INFO - downloader - skip downloading file 000005.jpg
2022-10-08 12:59:16,087 - INFO - downloader - skip downloading file 000006.jpg
2022-10-08 12:59:16,088 - INFO - dow

(2400, 64, 64, 3)
Epoch 1/40
86/86 [==============================] - 15s 168ms/step - loss: 0.6724 - accuracy: 0.5838
Epoch 2/40
86/86 [==============================] - 14s 162ms/step - loss: 0.5987 - accuracy: 0.6833
Epoch 3/40
86/86 [==============================] - 14s 168ms/step - loss: 0.5418 - accuracy: 0.7150
Epoch 4/40
86/86 [==============================] - 18s 205ms/step - loss: 0.5000 - accuracy: 0.7521
Epoch 5/40
86/86 [==============================] - 16s 186ms/step - loss: 0.4598 - accuracy: 0.7729
Epoch 6/40
86/86 [==============================] - 17s 201ms/step - loss: 0.4294 - accuracy: 0.7950
Epoch 7/40
86/86 [==============================] - 15s 180ms/step - loss: 0.3843 - accuracy: 0.8321
Epoch 8/40
86/86 [==============================] - 14s 161ms/step - loss: 0.3415 - accuracy: 0.8571
Epoch 9/40
86/86 [==============================] - 14s 161ms/step - loss: 0.3002 - accuracy: 0.8813
Epoch 10/40
86/86 [==============================] - 14s 165ms/step - los